# Functional test with FTDI - ADF4351
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from utilities.adapters import peripherals
from signal_generators.adf435x import ADF4351 

Populating the interactive namespace from numpy and matplotlib
No USB device matches URL ftdi://ftdi:ft232h/1


## Debug mode?

In [5]:
cls = ADF4351 

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
    _data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
    _ss = peripherals.Pin.get_Ftdi_pin(pin_id = 3)
    # _ss2 = peripherals.Pin.get_Ftdi_pin(pin_id = 0)

    _spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 1)
else:
    _spi = _ss = None  # using None for testing without actual hardware device.

adf = cls(_spi, _ss)


****** Virtual device. Data may not be real ! ******



## Current Configuration

In [7]:
RF_out_min = 2.2e9 / 64
RF_out_max = 4.4e9
RF_out_min, RF_out_max

(34375000.0, 4400000000.0)

In [8]:
adf.set_frequency(RF_out_min, channel_resolution = 100e3, rf_divider_as = None)
# adf.set_frequency(35e6)
# adf.set_frequency(3.9e9) 
adf.set_frequency(RF_out_max, channel_resolution = 100e3, rf_divider_as = None)

# adf.phaser.set_phase(90)

df_dividers, df_controls = adf.current_configuration
df_dividers

Failed in setting RF_out as 34375000.0 Hz = 34375000.0 Hz / 1 (freq_vco / rf_divider).
Failed in setting RF_out as 34375000.0 Hz = 68750000.0 Hz / 2 (freq_vco / rf_divider).
Failed in setting RF_out as 34375000.0 Hz = 137500000.0 Hz / 4 (freq_vco / rf_divider).
Failed in setting RF_out as 34375000.0 Hz = 275000000.0 Hz / 8 (freq_vco / rf_divider).
Failed in setting RF_out as 34375000.0 Hz = 550000000.0 Hz / 16 (freq_vco / rf_divider).
Failed in setting RF_out as 34375000.0 Hz = 1100000000.0 Hz / 32 (freq_vco / rf_divider).


,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,25000000
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.0,NaN,True,50000000
2,_R_Counter,_ReferenceDoubler,5.000000e+07,1.0,NaN,True,50000000
3,_ReferenceDivider,_R_Counter,5.000000e+07,2.0,NaN,True,25000000
4,_PhaseFrequencyDetector,_ReferenceDivider,2.500000e+07,1.0,NaN,True,25000000
5,_RF_N_Divider,_VCO,4.400000e+09,176.0,176.0,True,25000000
6,_VCO,_PhaseFrequencyDetector,2.500000e+07,176.0,NaN,True,4400000000
7,_RF_Divider,_VCO,4.400000e+09,1.0,NaN,True,4400000000
8,_RF_Output,_RF_Divider,4.400000e+09,1.0,NaN,True,4400000000
9,_AuxOutput,_VCO,4.400000e+09,1.0,NaN,True,4400000000


In [9]:
df_controls

,type,source_type,source_freq,phase_adjust_enabled,phase,prescaler,my_divider,is_integer,my_freq,mode,charge_pump_current,three_state_enabled,cancelation_enabled
0,_Phaser,_RF_N_Divider,2.500000e+07,False,1.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_Prescaler,_VCO,4.400000e+09,NaN,NaN,8/9,0.888889,False,3.911111e+09,NaN,NaN,NaN,NaN
2,_MuxOut,N_DIVIDER,2.500000e+07,NaN,NaN,NaN,1.000000,True,2.500000e+07,NaN,NaN,NaN,NaN
3,_BandSelectClockDivider,_ReferenceDivider,2.500000e+07,NaN,NaN,NaN,50.000000,True,5.000000e+05,NaN,NaN,NaN,NaN
4,_ClockDivider,_ReferenceDivider,2.500000e+07,NaN,NaN,NaN,4095.000000,True,6.105000e+03,NaN,NaN,NaN,NaN
5,_NoiseControl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW_NOISE_MODE,NaN,NaN,NaN
6,_ChargePump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.31,False,True


In [10]:
adf.freq_resolution

100000.0

## Find Integer N-dividers for a Frequency

In [11]:
adf.find_integer_N_dividers(freq_desired = 35e6,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

[((2, 5, 2, 448.0, 64),
  (25000000.0, 50000000, 10000000, 5000000, 2240000000, 35000000, 35000000)),
 ((2, 10, 2, 896.0, 64),
  (25000000.0, 50000000, 5000000, 2500000, 2240000000, 35000000, 35000000)),
 ((2, 20, 2, 1792.0, 64),
  (25000000.0, 50000000, 2500000, 1250000, 2240000000, 35000000, 35000000)),
 ((2, 25, 2, 2240.0, 64),
  (25000000.0, 50000000, 2000000, 1000000, 2240000000, 35000000, 35000000)),
 ((2, 40, 2, 3584.0, 64),
  (25000000.0, 50000000, 1250000, 625000, 2240000000, 35000000, 35000000)),
 ((2, 50, 2, 4480.0, 64),
  (25000000.0, 50000000, 1000000, 500000, 2240000000, 35000000, 35000000)),
 ((2, 80, 2, 7168.0, 64),
  (25000000.0, 50000000, 625000, 312500, 2240000000, 35000000, 35000000)),
 ((2, 100, 2, 8960.0, 64),
  (25000000.0, 50000000, 500000, 250000, 2240000000, 35000000, 35000000)),
 ((2, 125, 2, 11200.0, 64),
  (25000000.0, 50000000, 400000, 200000, 2240000000, 35000000, 35000000)),
 ((2, 160, 2, 14336.0, 64),
  (25000000.0, 50000000, 312500, 156250, 2240000000,

In [12]:
adf.set_dividers(*(2, 5, 2, 448.0, 64))
adf.current_configuration[0]

,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,2.500000e+07
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.0,NaN,True,5.000000e+07
2,_R_Counter,_ReferenceDoubler,5.000000e+07,5.0,NaN,True,1.000000e+07
3,_ReferenceDivider,_R_Counter,1.000000e+07,2.0,NaN,True,5.000000e+06
4,_PhaseFrequencyDetector,_ReferenceDivider,5.000000e+06,1.0,NaN,True,5.000000e+06
5,_RF_N_Divider,_VCO,2.240000e+09,448.0,448.0,True,5.000000e+06
6,_VCO,_PhaseFrequencyDetector,5.000000e+06,448.0,NaN,True,2.240000e+09
7,_RF_Divider,_VCO,2.240000e+09,64.0,NaN,True,3.500000e+07
8,_RF_Output,_RF_Divider,3.500000e+07,1.0,NaN,True,3.500000e+07
9,_AuxOutput,_VCO,2.240000e+09,1.0,NaN,True,2.240000e+09


In [13]:
adf.status

OrderedDict([('mclk',
              OrderedDict([('type', '_ReferenceInput'),
                           ('my_freq', 25000000.0)])),
             ('ref_doubler',
              OrderedDict([('type', '_ReferenceDoubler'),
                           ('source_type', '_ReferenceInput'),
                           ('source_freq', 25000000.0),
                           ('my_divider', 2),
                           ('is_integer', True),
                           ('my_freq', 50000000)])),
             ('r_counter',
              OrderedDict([('type', '_R_Counter'),
                           ('source_type', '_ReferenceDoubler'),
                           ('source_freq', 50000000),
                           ('my_divider', 5),
                           ('is_integer', True),
                           ('my_freq', 10000000)])),
             ('ref_divider',
              OrderedDict([('type', '_ReferenceDivider'),
                           ('source_type', '_R_Counter'),
                       

In [16]:
adf.print()


<< REGISTER_0 >>  :  ('0x580000', '0b10110000000000000000000')
[ Reserved_31 ] :  0
[ INT ]         :  176
[ FRAC ]        :  0
[ Index ]       :  0

<< REGISTER_1 >>      :  ('0x800fff9', '0b1000000000001111111111111001')
[ Reserved_29 ]     :  0
[ Phase_Adjust ]    :  0
[ Prescaler_Value ] :  1
[ Phase_Value ]     :  1
[ MOD ]             :  4095
[ Index ]           :  1

<< REGISTER_2 >>                   :  ('0x130041c2', '0b10011000000000100000111000010')
[ Reserved_31 ]                  :  0
[ Low_Noise_and_Low_Spur_Modes ] :  0
[ MUXOUT ]                       :  4
[ Reference_Doubler ]            :  True
[ RDIV2 ]                        :  True
[ R_Counter ]                    :  1
[ Double_Buffer ]                :  0
[ Charge_Pump_Current_Setting ]  :  0
[ LDF ]                          :  1
[ LDP ]                          :  1
[ Phase_Detector_Polarity ]      :  1
[ Power_Down ]                   :  0
[ Charge_Pump_Three_State ]      :  0
[ Counter_Reset ]                :

In [14]:
adf.init()

## Member functions test

In [15]:
for f in dir(cls):
    if not f.startswith('_'):
        print('adf.{}()'.format(f))

adf.DEBUG_MODE()
adf.DEBUG_MODE_PRINT_REGISTER()
adf.DEBUG_MODE_SHOW_BUS_DATA()
adf.FREQ_MCLK()
adf.FREQ_REF()
adf.N_OUTPUT_CLOCKS()
adf.apply_signal()
adf.close()
adf.current_configuration()
adf.current_frequency()
adf.current_phase()
adf.do()
adf.do_on_devices()
adf.enable()
adf.enable_output()
adf.enable_output_channel()
adf.enabled()
adf.find_integer_N_dividers()
adf.freq_pfd()
adf.freq_resolution()
adf.frequency()
adf.init()
adf.is_virtual_device()
adf.load_registers()
adf.pause()
adf.phase_resolution()
adf.power_downed()
adf.print()
adf.read_all_registers()
adf.registers_values()
adf.reset()
adf.resume()
adf.select_freq_source()
adf.select_phase_source()
adf.set_dividers()
adf.set_frequency()
adf.set_phase()
adf.shape()
adf.start()
adf.status()
adf.stop()
adf.update()
adf.write_all_registers()
